Base script for comparison existance of output-format in the prompt

In [1]:
# review text

import sys
from pathlib import Path
import pandas as pd

sys.path.append('../../')

import _sample_reviews
import _prompts

from langchain_community.llms import Ollama

# YOUR REVIEW TEXT
review_text = _sample_reviews.sample_11

# YOUR OLLAMA MODEL OF THIS SCRIPT
llm = Ollama(model='llama2:7b-chat-q4_0', temperature=0.4)

In [2]:
from datetime import datetime
from hashlib import sha224

# create an object with review_ID, review_text and the datetime of creation
def create_review(review_text, review_ID):    
    review_obj = {
        "review_ID": review_ID,
        "review_text": review_text,
        "datetime": datetime.now()
    }

    # create a hash of the review_text for creating unique collection with Chromadb
    hash = sha224(str(review_obj).encode()).hexdigest()
    review_obj['hash'] = hash

    return review_obj

# the temporary review_ID
review_ID = 123
# select a comment to test with
review_obj = create_review(review_text, review_ID)

print(review_obj)
print('\n\n')
print('Len of review_text:', len(review_text.split()))

{'review_ID': 123, 'review_text': 'overall good magic card game. the only issue i can really see is the random algorithem for shuffeling the deck, more times than not you either hit a major mana pocket or no mana at all for half the deck reguardless of deck size. i would also suggest to the devs to allow specific card buying for coins as its really hard to get a single card from an entire set of cards, and yes i would expect the cost to be much higher than for booster packs, adjusted for card value and rareity.', 'datetime': datetime.datetime(2024, 4, 4, 11, 36, 46, 728435), 'hash': 'b91c83d28c182e83b73881aee70df5c6aef7f7735165430366086828'}



Len of review_text: 94


In [3]:
# define callbacks for detecting token usage
# copied from ../previous_tests/ollama_get_token_usage_example.ipynb

from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.outputs.llm_result import LLMResult
from collections import deque

class TokenUsageCallbackHandler(BaseCallbackHandler):

    def __init__(self, deque: deque = None):
        super().__init__()
        self.deque = deque

    def on_llm_end(self, response: LLMResult, **kwargs) -> None:

        generation = response.generations[0][0]
        gen_info = generation.generation_info

        # get token usage
        # ref: https://github.com/orgs/langfuse/discussions/1179
        token_usage = gen_info.get('prompt_eval_count', 0) + gen_info.get('eval_count', 0)
        # get time costed (local machine)
        # instead of getting total duration, we get the prompt_eval_duration and eval_duration to exclude the load duration (e.g. to load the model to the gpu, etc.)
        time_costed = gen_info.get('prompt_eval_duration', 1e-10) + gen_info.get('eval_duration', 1e-10)     # in ns, a small value to indicate a inf time when it fails


        # create an object to store the token usage and time costed
        token_usage_obj = {
            'token_usage': token_usage,
            'time_costed': time_costed
        }

        # append the object to the deque
        self.deque.append(token_usage_obj)



common_deque = deque()
chain_config = {
    "callbacks": [TokenUsageCallbackHandler(common_deque)],
}

In [4]:
import chromadb

# use docker
# chroma_client = chromadb.HttpClient(host="localhost", port=8000)

In [5]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter, RecursiveCharacterTextSplitter
from transformers import AutoTokenizer
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.vectorstores import Chroma

# also change this to use Mistral Embeddings
embedding_func = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

text_spliter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base', chunk_size=250, chunk_overlap=40
)

docs = text_spliter.create_documents(
    [review_text], metadatas=[{"source":"review_01"}]
)

temp_db = Chroma.from_documents(
    documents=docs,
    embedding=embedding_func,
    collection_name=review_obj['hash']
)

retriever = temp_db.as_retriever(search_kwargs={"k": 5})

/Users/michaelcheng/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/michaelcheng/miniforge3/envs/fyp-test-tm/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


---

prompting

In [6]:
# global constants
GAME_ASPECTS = ['Gameplay', 'Narrative', 'Accessibility', 'Sound', 'Graphics & Art Design', 'Performance', 'Bug', 'Suggestion', 'Price', 'Overall']

3-3-4

with one-shot output format provided

In [7]:
# 3-3-4
# modified from main branch

from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

token_usage_json_list = []
embedding_usage_info_list = []


for (start, end) in [(0, 3), (3, 6), (6, 10)]:
    aspects = GAME_ASPECTS[start:end]

    my_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)


    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": aspects,
        "output_format": output_format,
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Here is the JSON response:
{"Gameplay": "...", "Narrative": "...", "Accessibility": "..."}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Here is the JSON response:
{"Sound": "...", "Graphics & Art Design": "...", "Performance": "..."}
--------------------
Here is the JSON output:

{"Bug": "...", "Suggestion": "...", "Price": "...", "Overall": "..."}
--------------------


In [8]:
token_usage_json_list

[{'token_usage': 356, 'time_costed': 1118367000},
 {'token_usage': 309, 'time_costed': 987446000},
 {'token_usage': 326, 'time_costed': 1153518000}]

3-3-4 without output format provided

In [9]:
# without output_format

token_usage_json_list = []
embedding_usage_info_list = []


for (start, end) in [(0, 3), (3, 6), (6, 10)]:
    aspects = GAME_ASPECTS[start:end]

    my_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)


    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": aspects,
        # "output_format": output_format,
        "output_format": '',
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Here is the JSON response:
{
"Gameplay": "Good magic card game.",
"Narrative": "NA",
"Accessibility": "NA"
}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Here is the JSON response:
{
"Sound": "The sound effects and music are well-done and add to the overall atmosphere of the game.",
"Graphics & Art Design": "The art design and visuals are impressive and help bring the game world to life.",
"Performance": "The random deck shuffling algorithm can be frustrating at times, leading to inconsistent mana distribution."
}
--------------------
Here is the JSON response:
{
"Bug": "Random algorithm for shuffling the deck can lead to uneven distribution of mana.",
"Suggestion": "Allow specific card buying for coins, adjusted for card value and rarity.",
"Price": "NA",
"Overall": "Good magic card game with some issues."
}
--------------------


In [10]:
token_usage_json_list

[{'token_usage': 260, 'time_costed': 1077267000},
 {'token_usage': 311, 'time_costed': 1986945000},
 {'token_usage': 306, 'time_costed': 1857688000}]

---

2-2-2-2-2 prompting

with output_format provided

In [11]:
token_usage_json_list = []
embedding_usage_info_list = []


for (start, end) in [(0, 2), (2, 4), (4, 6), (6, 8), (8, 10)]:
    aspects = GAME_ASPECTS[start:end]

    my_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)


    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": aspects,
        "output_format": output_format,
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Here is the JSON response:
{"Gameplay": "...", "Narrative": "..."}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{"Accessibility": "...", "Sound": "..."}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{"Graphics & Art Design": "...", "Performance": "..."}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Here is the JSON response:
{"Bug": "...", "Suggestion": "..."}
--------------------
Here is the JSON output for the reviews:

{"Price": "...", "Overall": "..."}
--------------------


In [12]:
token_usage_json_list

[{'token_usage': 289, 'time_costed': 918386000},
 {'token_usage': 274, 'time_costed': 727486000},
 {'token_usage': 286, 'time_costed': 779203000},
 {'token_usage': 289, 'time_costed': 879424000},
 {'token_usage': 285, 'time_costed': 911065000}]

2-2-2-2-2 wihtout output format

In [13]:
token_usage_json_list = []
embedding_usage_info_list = []


for (start, end) in [(0, 2), (2, 4), (4, 6), (6, 8), (8, 10)]:
    aspects = GAME_ASPECTS[start:end]

    my_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)


    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": aspects,
        "output_format": '',
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{
"Gameplay": "Good magic card game with some issues with deck shuffling algorithm."
"Narrative": "No major narrative issues."
}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Here is the JSON output:
{
"Accessibility": "The random algorithm for shuffling the deck can be frustrating, often leading to major mana pockets or no mana at all for half of the deck regardless of deck size.",
"Sound": "NA"
}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Here is the JSON response:
{
"Graphics & Art Design": "The game features visually appealing graphics and art design.",
"Performance": "The random algorithm for shuffling the deck can be unpredictable."
}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Here is the JSON response:
{
"Bug": "The random algorithm for shuffling the deck can lead to major mana pockets or no mana at all for half of the deck, regardless of deck size.",
"Suggestion": "Allow specific card buying for coins, as it is difficult to obtain a single card from an entire set of cards and the cost should be higher than for booster packs, adjusted for card value and rarity."
}
--------------------
Here is the JSON response:
{
"Price": "The game's price is reasonable for what it offers, but the random shuffling algorithm can be frustrating at times."
"Overall": "Overall, the game is good, but the shuffling algorithm and limited card buying options are drawbacks."
}
--------------------


In [14]:
token_usage_json_list

[{'token_usage': 254, 'time_costed': 1042523000},
 {'token_usage': 278, 'time_costed': 1480756000},
 {'token_usage': 270, 'time_costed': 1298514000},
 {'token_usage': 322, 'time_costed': 2236811000},
 {'token_usage': 289, 'time_costed': 1721857000}]

---

5-5 with output format provided

In [15]:
token_usage_json_list = []
embedding_usage_info_list = []


for (start, end) in [(0, 5), (5, 10)]:
    aspects = GAME_ASPECTS[start:end]

    my_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)


    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": aspects,
        "output_format": output_format,
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Here is the JSON output:

{
"Gameplay": "...",
"Narrative": "...",
"Accessibility": "...",
"Sound": "...",
"Graphics & Art Design": "NA"
}
--------------------
Here is the JSON output:

{"Performance": "...", "Bug": "...", "Suggestion": "...", "Price": "...", "Overall": "..."}
--------------------


In [16]:
token_usage_json_list

[{'token_usage': 363, 'time_costed': 1489767000},
 {'token_usage': 346, 'time_costed': 1295015000}]

5-5 without output format provided

In [17]:
token_usage_json_list = []
embedding_usage_info_list = []


for (start, end) in [(0, 5), (5, 10)]:
    aspects = GAME_ASPECTS[start:end]

    my_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)


    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": aspects,
        "output_format": '',
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Here is the JSON response:
{
"Gameplay": "Good magic card game.",
"Narrative": "NA",
"Accessibility": "NA",
"Sound": "NA",
"Graphics & Art Design": "NA"
}
--------------------
Here is the JSON response:

{
"Performance": "The game runs smoothly overall, but the shuffling algorithm can be unpredictable.",
"Bug": "Occasionally experiences issues with deck shuffling.",
"Suggestion": "Allow specific card buying for coins to make it easier to obtain desired cards.",
"Price": "The cost of cards should be adjusted based on rarity and value.",
"Overall": "Good magic card game with some minor issues."
}
--------------------


In [18]:
token_usage_json_list

[{'token_usage': 286, 'time_costed': 1447883000},
 {'token_usage': 342, 'time_costed': 2474479000}]

---

1 aspect per prompt, with output format

In [19]:
for aspect in GAME_ASPECTS:
    my_question = _prompts.QUESTION_TEMPLATE_01 + f"is {aspect}"
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str([aspect])[1:-1].replace('\'', '\"'), output_json_template=str({aspect: '...'}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)

    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": [aspect],
        "output_format": output_format,
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{"Gameplay": "...the only issue i can really see is the random algorithm for shuffling the deck..."}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{"Narrative": "..."}
--------------------


...

{"Bug": "...the random algorithm for shuffling the deck, more times than not you either hit a major mana pocket or no mana at all for half the deck regardless of deck size."}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{"Suggestion": "...more times than not you either hit a major mana pocket or no mana at all for half the deck reguardless of deck size..."}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{"Price": "..."}
--------------------
{"Overall": "...good magic card game. The only issue I can really see is the random algorithm for shuffling the deck, more times than not you either hit a major mana pocket or no mana at all for half the deck regardless of deck size."}
--------------------


In [20]:
token_usage_json_list

[{'token_usage': 286, 'time_costed': 1447883000},
 {'token_usage': 342, 'time_costed': 2474479000},
 {'token_usage': 276, 'time_costed': 933948000},
 {'token_usage': 262, 'time_costed': 615498000},
 {'token_usage': 298, 'time_costed': 1273970000},
 {'token_usage': 254, 'time_costed': 578788000},
 {'token_usage': 266, 'time_costed': 679186000},
 {'token_usage': 290, 'time_costed': 1165529000},
 {'token_usage': 295, 'time_costed': 1256159000},
 {'token_usage': 292, 'time_costed': 1120975000},
 {'token_usage': 254, 'time_costed': 584753000},
 {'token_usage': 308, 'time_costed': 1446682000}]

1 aspect per prompt, without output format

In [21]:
for aspect in GAME_ASPECTS:
    my_question = _prompts.QUESTION_TEMPLATE_01 + f"is {aspect}"
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str([aspect])[1:-1].replace('\'', '\"'), output_json_template=str({aspect: '...'}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)

    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": [aspect],
        "output_format": '',
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Here is the JSON response:
{
"Gameplay": "Overall good magic card game with some issues with deck shuffling algorithm."
}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Here is the JSON response:
{
"Narrative": "The game has a good magic card gameplay, but the algorithm for shuffling the deck can be unpredictable and lead to unfair advantages or disadvantages."
}
--------------------


...

Here is the JSON response:
{
"Bug": "Random algorithm for shuffling the deck can lead to unbalanced mana distribution, making it difficult to play certain cards."
}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Here is the JSON response:
{
"Suggestion": "Allow specific card buying for coins."
}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Here is the JSON response:
{
"Price": "The game's price is reasonable for what it offers, but the random deck shuffling algorithm can lead to frustrating experiences."
}
--------------------
Here is the JSON output for the aspect "Overall":

{
"Overall": "Good magic card game with some issues with deck shuffling algorithm."
}
--------------------
